# Importing Datasets

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
#lmao

In [2]:
#importing all datasets to check what we have

gdp_and_income = pd.read_csv("../data/gdp_and_income_ca.csv")
act_2019_ca = pd.read_csv("../data/act_2019_ca.csv")
ada_expenses = pd.read_excel('../data/currentexpense1920.xlsx')
teacher_salaries = pd.read_csv('../data/teacher_salary_ca.csv')

### Helper Methods

In [3]:
def post_merge_clean(df, index):
    """
    This function takes the post merge df and
    1. Drops the index_x and index_y columns
    2. Renames key_0 to original index name
    """
    df.drop([f'{index}_x',f'{index}_y'],
            axis=1, 
            inplace=True)
    
    df.rename(columns = {
    'key_0': f'{index}'
    },
    inplace = True)
    
    return df

In [4]:
def clean_left_merge(left, right, index):
    """
    Runs a left join merge on index and then calls
    post_merge_clean
    to clean up excess columns created during merge.
    
    Assumes index name exists in both dfs
    """
    left = left.merge(
    right=right,
    how='left',
    left_on = left[index],
    right_on = right[index])
    
    post_merge_clean(left, index)
    
    return left

# Cleaning California Income

## Splicing New Dataframes

Because of how California Income is formatted, best to be split into three seperate tables, cleaned, and merged later

In [5]:
personal_income_ca = gdp_and_income[gdp_and_income['LineCode'] == 1]
population_ca = gdp_and_income[gdp_and_income['LineCode'] == 2]
per_capita_income_ca = gdp_and_income[gdp_and_income['LineCode'] == 3]

## Dropping

Removing unncessary columns

In [6]:
per_capita_income_ca.drop(columns=['GeoFips','LineCode','Description'],inplace=True,axis=1)

In [7]:
population_ca.drop(columns=['GeoFips','LineCode','Description'],inplace=True,axis=1)

In [8]:
personal_income_ca.drop(columns=['GeoFips','LineCode','Description'],inplace=True,axis=1)

## Renaming

Renaming useful columns

In [9]:
per_capita_income_ca.rename(columns = {
    'GeoName':'county',
    '2019':'per_capita_income'
    },
    inplace = True)

In [10]:
population_ca.rename(columns = {
    'GeoName':'county',
    '2019':'population'
    },
    inplace = True)

In [11]:
personal_income_ca.rename(columns = {
    'GeoName':'county',
    '2019':'personal_income'
    },
    inplace = True)

## Merging

Merging the three dataframes into one original one

In [12]:
personal_income_ca = clean_left_merge(personal_income_ca, population_ca, 'county')

In [13]:
personal_income_ca = clean_left_merge(personal_income_ca, per_capita_income_ca, 'county')

## Formatting Data

Some data needs to be formatted correctly to be mergeable.

* i.e. 'Los Angeles, CA' -> 'Los Angeles'

In [14]:
personal_income_ca['county'] = personal_income_ca['county'].map(lambda string: string[:-4] if string[-4:] == ', CA' else string)

# Cleaning ACT

## Dropping

In [15]:
act_2019_ca.drop(['CDS','CCode','SCode','SName','Unnamed: 17','Year'], axis=1, inplace=True)

## Renaming

In [16]:
act_2019_ca.rename(columns={
    'RType':'report_type',
    'CName':'county',
    'CDCode':'cds_code',
    'Enroll12':'enrolled_12',
    'NumTstTakr':'num_test_taker',
    'AvgScrRead':'avg_act_reading',
    'AvgScrEng':'avg_act_english',
    'AvgScrMath':'avg_act_math',
    'AvgScrSci':'avg_act_sci',
    'NumGE21':'num_over_21_score',
    'PctGE21':'percent_over_21_score',
    'DName':'ts1_dname'
}, inplace=True)

## Null Values

In [17]:
for column in act_2019_ca:
    act_2019_ca[column] = act_2019_ca[column].map(lambda cell: np.nan if cell == '*' else cell)

In [18]:
act_2019_ca.dropna(inplace=True)

## Merging

In [19]:
act_2019_ca = clean_left_merge(act_2019_ca, personal_income_ca, 'county')

## Formatting

In [20]:
act_2019_ca['cds_code'] = act_2019_ca['cds_code'].map(lambda code: str(code)[2:-2])

# School Expenses

## Renaming

In [21]:
ada_expenses.rename(columns={
    'CO':'county_code',
    'CDS':'cds_code',
    'DISTRICT':'district',
    'EDP 365':'edp_365',
    'Current\nExpense ADA':'current_expense_ada',
    'Current\nExpense Per ADA':'current_expense_per_ada',
    'LEA_Type':'lea_type'
}, inplace=True)

## Formating Data

In [22]:
ada_expenses['cds_code'] = ada_expenses['cds_code'].map(lambda code: str(code))

## Merging

In [23]:
act_2019_ca = clean_left_merge(act_2019_ca,ada_expenses,'cds_code')

# Teacher Salaries

In [24]:
teacher_salaries.rename(columns=
    {'District':'district',
    'Teachers':'teachers',
    '2013-14':'avg_teacher_salary_13_14',
     '2017-18':'avg_teacher_salary_17_18',
    '2018-19':'avg_teacher_salary_18_19'},
                       inplace=True)

In [25]:
teacher_salaries['district'].value_counts

<bound method IndexOpsMixin.value_counts of 0      Alameda County Office of Ed. (Alameda)
1                 New Haven Unified (Alameda)
2                Pleasanton Unified (Alameda)
3                   Hayward Unified (Alameda)
4                   Fremont Unified (Alameda)
                        ...                  
823           Marysville Joint Unified (Yuba)
824                     Wheatland High (Yuba)
825                  Plumas Elementary (Yuba)
826          Yuba County Office of Ed. (Yuba)
827            Camptonville Elementary (Yuba)
Name: district, Length: 828, dtype: object>

In [26]:
teacher_salaries['district'] = teacher_salaries['district'].map(lambda name: name[:name.index('(')-1])

In [27]:
final = clean_left_merge(act_2019_ca,teacher_salaries,'district')

In [28]:
final.drop(['ts1_dname'],axis=1,inplace=True)

In [29]:
final.dropna(inplace=True)

## Exporting

In [30]:
final.to_csv('../data/processed/final.csv',index=False)